In [2]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")

In [ ]:
model.train(
    data= r"C:\path\to\data.yaml", #actual path of data.yml file
    epochs=10,
    imgsz=640,
    batch=16,
    name="soyabean_disease_detector"
)

New https://pypi.org/project/ultralytics/8.3.170 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.192  Python-3.9.23 torch-2.1.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=E:\PROJECTS\UAV_Based_Disease_Detection\uav_leaf_yolo_dataset\data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=soyabean_disease_detector, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stri

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021137301D00>
fitness: 0.03008735536808268
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([   0.035961,    0.017349,    0.026655])
names: {0: 'Healthy_Soyabean', 1: 'Soyabean_Mosaic', 2: 'Soyabean_Rust'}
plot: True
results_dict: {'metrics/precision(B)': 0.060539287914914135, 'metrics/recall(B)': 0.28516694033935414, 'metrics/mAP50(B)': 0.06097530669321719, 'metrics/mAP50-95(B)': 0.026655360776401064, 'fitness': 0.03008735536808268}
save_dir: WindowsPath('runs/detect/soyabean_disease_detector6')
speed: {'preprocess': 2.2432804107666016, 'inference': 360.1231575012207, 'loss': 0.0, 'postprocess': 226.27035776774088}

In [ ]:
from ultralytics import YOLO

# Load your trained YOLOv8 model
model = YOLO("/path/to/runs/detect/soyabean_disease_detector6/weights/best.pt")

# Run inference on an image
results = model("/path/to/image_for_testing.jpg", save=True)





image 1/1 e:\PROJECTS\UAV_Based_Disease_Detection\Soyabean_UAV-Based_Image_Dataset\Soyabean_Mosaic\Soyabean_Mosaic\DJI_20231215155242_0106_D_000008.jpg: 384x640 3 Soyabean_Rusts, 214.4ms
Speed: 3.1ms preprocess, 214.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict


In [ ]:
from ultralytics import YOLO
import os
import cv2
import random
import numpy as np
from PIL import Image, ImageDraw, ImageFont

# Load YOLOv8 model
model = YOLO("path/to/runs/detect/soyabean_disease_detector6/weights/best.pt")

# Fixed color for each label
label_colors = {label: tuple(random.randint(0, 255) for _ in range(3)) for label in model.names.values()}

def generate_disease_table_overlay(unique_diseases):
    row_height = 32
    padding = 8
    col1 = 100  # Disease name
    col2 = 60   # Confidence
    col3 = 40   # Color box
    table_width = col1 + col2 + col3 + 4 * padding
    table_height = (len(unique_diseases) + 1) * row_height + 2 * padding

    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    image = Image.new("RGBA", (table_width, table_height), (255, 255, 255, 200))  # semi-transparent white
    draw = ImageDraw.Draw(image)

    # Header
    draw.text((padding, padding), "Disease", font=font, fill="black")
    draw.text((padding + col1, padding), "Conf", font=font, fill="black")
    draw.text((padding + col1 + col2, padding), "Box", font=font, fill="black")

    for i, (disease, (conf, color)) in enumerate(unique_diseases.items()):
        y = padding + (i + 1) * row_height
        draw.text((padding, y), disease, font=font, fill="black")
        draw.text((padding + col1, y), f"{conf}", font=font, fill="black")
        draw.rectangle(
            [(padding + col1 + col2 + 5, y + 5), (padding + col1 + col2 + 35, y + 25)],
            fill=color,
            outline="black"
        )

    return image

def detect_diseases_in_image(image_path, conf_threshold=0.1, output_path="final_output.jpg"):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    results = model(image_path, conf=conf_threshold)
    boxes = results[0].boxes
    unique_diseases = {}

    if boxes is None or len(boxes) == 0:
        print("⚠️ No diseases detected.")
        return []

    image = cv2.imread(image_path)

    for box in boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        label = model.names[cls_id]

        if label == "Healthy_Soyabean":
            continue

        x1, y1, x2, y2 = map(int, box.xyxy[0])
        color = label_colors[label]
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        if label not in unique_diseases or conf > unique_diseases[label][0]:
            unique_diseases[label] = (round(conf, 2), color)

    # Generate small overlay legend
    table_overlay = generate_disease_table_overlay(unique_diseases)

    # Convert image to RGBA to overlay
    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).convert("RGBA")
    iw, ih = image_pil.size
    tw, th = table_overlay.size

    # Paste table onto top-right corner
    image_pil.paste(table_overlay, (iw - tw - 10, 10), table_overlay)

    # Save final result
    final_image = cv2.cvtColor(np.array(image_pil.convert("RGB")), cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, final_image)
    print(f"✅ Final image with top-right table saved to {output_path}")

    return [(label, conf) for label, (conf, _) in unique_diseases.items()]


In [13]:
#image_path = "Soyabean_UAV-Based_Image_Dataset/Soyabean_Mosaic/Soyabean_Mosaic/DJI_20231215155242_0106_D_000008.jpg"
image_path = "Soyabean_UAV-Based_Image_Dataset/Soyabean_Mosaic/Soyabean_Mosaic/DJI_20231215160828_0110_D_000080.jpg"

detections = detect_diseases_in_image(image_path)

for i, (label, conf) in enumerate(detections, 1):
    print(f"{i}. Disease: {label}, Confidence: {conf * 100:.1f}%")



image 1/1 e:\PROJECTS\UAV_Based_Disease_Detection\Soyabean_UAV-Based_Image_Dataset\Soyabean_Mosaic\Soyabean_Mosaic\DJI_20231215160828_0110_D_000080.jpg: 384x640 2 Healthy_Soyabeans, 1 Soyabean_Mosaic, 23 Soyabean_Rusts, 201.7ms
Speed: 7.0ms preprocess, 201.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


✅ Final image with top-right table saved to final_output.jpg
1. Disease: Soyabean_Rust, Confidence: 34.0%
2. Disease: Soyabean_Mosaic, Confidence: 13.0%


In [ ]:
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO("path/to/runs/detect/soyabean_disease_detector6/weights/best.pt")

# Evaluate on the test set
metrics = model.val(data= r"C:\path\to\data.yaml", split='val', save_json=True)

# Extract per-class lists
precision_list = metrics.box.p       # list
recall_list = metrics.box.r          # list
f1_list = metrics.box.f1             # list

# Scalars (not lists)
map50 = metrics.box.map50            # scalar
map = metrics.box.map                # scalar

# Get class names
class_names = model.names

# ✅ Print per-class metrics
print("\n📊 Evaluation Metrics per Class:")
print(f"{'Class':<25} {'Precision':<10} {'Recall':<10} {'F1 Score':<10}")
for i in range(len(precision_list)):
    print(f"{class_names[i]:<25} "
          f"{precision_list[i]:<10.3f} "
          f"{recall_list[i]:<10.3f} "
          f"{f1_list[i]:<10.3f}")

# ✅ Print overall metrics
print("\n📈 Overall Metrics:")
print(f"mAP@0.5      : {map50:.4f}")
print(f"mAP@0.5:0.95 : {map:.4f}")

# Optional: average precision, recall, F1
avg_precision = sum(precision_list) / len(precision_list)
avg_recall = sum(recall_list) / len(recall_list)
avg_f1 = sum(f1_list) / len(f1_list)
print(f"Avg Precision: {avg_precision:.4f}")
print(f"Avg Recall   : {avg_recall:.4f}")
print(f"Avg F1 Score : {avg_f1:.4f}")


Ultralytics YOLOv8.0.192  Python-3.9.23 torch-2.1.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs
val: Scanning E:\PROJECTS\UAV_Based_Disease_Detection\uav_leaf_yolo_dataset\valid\labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]WARNING  NMS time limit 0.650s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]
                   all          3        150     0.0605      0.285      0.061     0.0267
      Healthy_Soyabean          3         63     0.0873      0.317     0.0857      0.036
       Soyabean_Mosaic          3         87     0.0337      0.253     0.0363     0.0173
Speed: 1.4ms preprocess, 339.0ms inference, 0.0ms loss, 218.8ms postprocess 


📊 Evaluation Metrics per Class:
Class                     Precision  Recall     F1 Score  
Healthy_Soyabean          0.087      0.317      0.137     
Soyabean_Mosaic           0.034      0.253      0.060     

📈 Overall Metrics:
mAP@0.5      : 0.0610
mAP@0.5:0.95 : 0.0267
Avg Precision: 0.0605
Avg Recall   : 0.2852
Avg F1 Score : 0.0983
